# ETL (Extrair, Transformar e Carregar) da camada Silver para Gold.

Este notebook realiza o processo de ETL para transformar e carregar os dados da camada Silver para a camada Gold no data lake. A camada Gold é otimizada para consultas analíticas e relatórios, garantindo que os dados estejam prontos para uso por ferramentas de BI e análise avançada.


### Configuração do Ambiente de Desenvolvimento

In [1]:
!pip install pyspark
!pip install psycopg2
!pip install pandas

# Criação e Carga das Dimensões (Star Schema)

Esta seção tem como objetivo realizar a **criação e carga das tabelas dimensão**
da camada Gold, seguindo o modelo **Star Schema**, a partir de dados previamente
tratados na camada Silver.


### Importação de Bibliotecas

Nesta seção são importadas as bibliotecas necessárias para:
- manipulação de dados com PySpark
- criação de colunas derivadas
- conexão com o banco PostgreSQL

In [2]:
import pandas as pd
import psycopg2

from psycopg2.extras import execute_batch
from psycopg2 import extras

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, concat_ws, year, month, dayofmonth, quarter, 
    dayofweek, when, date_format, monotonically_increasing_id,
    lit, trim, upper
)

from pyspark.sql import functions as F

from pyspark.sql.types import StringType, IntegerType, DateType, DecimalType

### Configuração do Ambiente

Nesta etapa são configurados:
- a sessão Spark
- a conexão JDBC com o PostgreSQL
- o schema de destino da camada Gold

In [3]:
spark = SparkSession.builder \
    .appName("SilverToGold") \
    .getOrCreate()

POSTGRES_HOST = "localhost"
POSTGRES_PORT = "5432"
POSTGRES_DB = "cat_db"
POSTGRES_USER = "admin"
POSTGRES_PASSWORD = "admin"

conn_params = {
    "host": POSTGRES_HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "port": POSTGRES_PORT
}

connection_properties = {
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "driver": "org.postgresql.Driver"
}

try:
    conn = psycopg2.connect(**conn_params)
    
    query = "SELECT * FROM ACIDENTE"
    
    pdf = pd.read_sql(query, conn)
    df_silver = spark.createDataFrame(pdf)
    
    print("Dados da camada Silver carregados")
    df_silver.show()

except Exception as e:
    print(f"Erro na leitura: {e}")
finally:
    if conn:
        conn.close()
        
GOLD_SCHEMA = "gold"

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/17 15:14:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/17 15:14:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/tmp/ipykernel_24707/1149729806.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, conn)


Dados da camada Silver carregados


26/01/17 15:15:02 WARN TaskSetManager: Stage 0 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
Traceback (most recent call last):                                  (0 + 1) / 1]
  File "/home/yabamiah/Sandbox/cat-analytics/.venv/lib/python3.14/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
  File "/home/yabamiah/Sandbox/cat-analytics/.venv/lib/python3.14/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
    ~~~~~~~~~~~~~^^
BrokenPipeError: [Errno 32] Broken pipe
                                                                                

+------------------------+------------------------+--------------------+--------------------+---------------+-------------------------+--------------------+--------------------+--------------------+---------+-------------+---------------------+-----------------------+---------------+----------------+
|agente_causador_acidente|data_acidente_referencia|cbo_codigo_descricao|              cid_10|cnae_empregador|cnae_empregador_descricao|municipio_empregador|      natureza_lesao|parte_corpo_atingida|     sexo|tipo_acidente|uf_municipio_acidente|uf_municipio_empregador|data_nascimento|data_emissao_cat|
+------------------------+------------------------+--------------------+--------------------+---------------+-------------------------+--------------------+--------------------+--------------------+---------+-------------+---------------------+-----------------------+---------------+----------------+
|    Rua e Estrada - S...|              2023-03-27|       Administrador|   Frat da Clavicula| 

### Função Genérica de Carga de Dimensões

Esta função padroniza o processo de carga das dimensões, garantindo:
- remoção de duplicidades
- preservação da business key
- inserção no schema Gold
- validação básica pós-carga

In [4]:
def load_dim_with_psycopg2(table_name, pg_conn_params, spark):
    query = f"SELECT * FROM {table_name}"

    with psycopg2.connect(**pg_conn_params) as conn:
        pdf = pd.read_sql(query, conn)

    df_dim_loaded = spark.createDataFrame(pdf)
    return df_dim_loaded

In [5]:
def save_dimension(
    df_dim,
    dim_name,
    id_col_silver,
    other_cols,
    pg_conn_params,
    cols_to_drop=None
):
    """
    Salva dados em uma dimensão no PostgreSQL usando psycopg2,
    preservando a business key.
    Retorna um DataFrame Spark da dimensão carregada (com surrogate keys).
    """

    table_name = f"{GOLD_SCHEMA}.dim_{dim_name}"
    business_key_col = f"chv_{dim_name}_org"

    # Preparação da dimensão no Spark
    df_dim_unique = (
        df_dim
        .select(col(id_col_silver).alias(business_key_col), *other_cols)
        .distinct()
        .dropna(subset=[business_key_col])
    )

    if cols_to_drop:
        df_dim_unique = df_dim_unique.drop(*cols_to_drop)

    print(f"\n---> Criando e carregando Dimensão: {table_name}")
    count_unique = df_dim_unique.count()
    print(f"     Registros únicos: {count_unique}")

    if count_unique == 0:
        print("DataFrame vazio")
        return None

    # Converter para Pandas para inserção via psycopg2
    pdf = df_dim_unique.toPandas()

    columns = list(pdf.columns)
    cols_sql = ", ".join(columns)
    values_sql = ", ".join([f"%({c})s" for c in columns])

    insert_sql = f"""
        INSERT INTO {table_name} ({cols_sql})
        VALUES ({values_sql})
        ON CONFLICT ({business_key_col}) DO NOTHING
    """

    try:
        # Conexão PostgreSQL
        with psycopg2.connect(**pg_conn_params) as conn:
            with conn.cursor() as cur:
                execute_batch(
                    cur,
                    insert_sql,
                    pdf.to_dict(orient="records"),
                    page_size=1000
                )
            conn.commit()

        print("Inserção concluída")

        # Recarregar dimensão com surrogate keys via Spark
        df_dim_loaded = load_dim_with_psycopg2(
            table_name=table_name,
            pg_conn_params=pg_conn_params,
            spark=spark
        )


        count_check = df_dim_loaded.count()

        if count_check >= count_unique:
            print(f"Dimensão carregada. Registros confirmados: {count_check}")
        else:
            print(f"Confirmados ({count_check}) < esperados ({count_unique})")

        return df_dim_loaded

    except Exception as e:
        print(f"Erro ao gravar dimensão {table_name}: {e}")
        raise

### Dimensão Tempo

A dimensão tempo é utilizada para:
- data do acidente
- data de emissão da CAT
- data de nascimento do trabalhador

Ela permite análises temporais como sazonalidade, tendências e comparações anuais.

In [6]:
def create_dim_tempo(df_silver):    
    df_datas = df_silver.select("data_acidente_referencia").distinct() \
        .union(df_silver.select("data_emissao_cat").distinct()) \
        .union(df_silver.select("data_nascimento").distinct()) \
        .withColumnRenamed("data_acidente_referencia", "data") \
        .distinct() \
        .dropna()
    
    df_tempo = df_datas.select(
        col("data"),
        dayofmonth("data").alias("dia"),
        month("data").alias("mes"),
        date_format("data", "MMMM").alias("nome_mes"),
        quarter("data").alias("trimestre"),
        year("data").alias("ano"),
        dayofweek("data").alias("dia_semana"),
        when(dayofweek("data").isin(1, 7), True).otherwise(False).alias("is_fim_semana")
    )
    
    return save_dimension(
        df_tempo,
        "tempo",
        "data",
        ["dia", "mes", "nome_mes", "trimestre", "ano", "dia_semana", "is_fim_semana"],
        conn_params
    )

### Dimensão Trabalhador

Representa características demográficas e ocupacionais do trabalhador
no momento do acidente.

In [7]:
def create_dim_trabalhador(df_silver):
    df_trabalhador = df_silver.select(
        concat_ws("_", 
                  upper(trim(col("sexo"))),
                  col("cbo_codigo_descricao"),
                  col("data_nascimento")
        ).alias("id_trabalhador"),
        upper(trim(col("sexo"))).alias("sexo"),
        col("cbo_codigo_descricao").alias("fk_cbo"),
        col("data_nascimento").alias("fk_tempo_nascimento")
    )
    
    return save_dimension(
        df_trabalhador,
        "trabalhador",
        "id_trabalhador",
        ["sexo", "fk_cbo", "fk_tempo_nascimento"],
        conn_params
    )

### Dimensão Empregador

A dimensão empregador representa as características da empresa
responsável pelo vínculo de trabalho no momento do acidente.

Esta dimensão referencia:
- a dimensão CNAE (atividade econômica)
- a dimensão Município (localização do empregador)

Sua criação ocorre após a carga das dimensões independentes,
garantindo integridade referencial.

In [8]:
def create_dim_empregador(df_silver):
    df_empregador = df_silver.select(
        concat_ws("_", 
                  col("cnae_empregador"),
                  col("municipio_empregador")
        ).alias("id_empregador"),
        col("cnae_empregador").alias("fk_cnae"),
        col("municipio_empregador").alias("fk_municipio")
    )
    
    return save_dimension(
        df_empregador,
        "empregador",
        "id_empregador",
        ["fk_cnae", "fk_municipio"],
        conn_params
    )

### Dimensões de Classificação

As dimensões a seguir representam classificações oficiais utilizadas
para padronização e análise estatística.

In [9]:
def create_dim_cbo(df_silver):
    df_cbo = df_silver.select(
        col("cbo_codigo_descricao").alias("id_cbo"),
        col("cbo_codigo_descricao").alias("codigo"),
        col("cbo_codigo_descricao").alias("descricao")
    )
    
    return save_dimension(
        df_cbo,
        "cbo",
        "id_cbo",
        ["codigo", "descricao"],
        conn_params
    )


def create_dim_cnae(df_silver):
    df_cnae = df_silver.select(
        col("cnae_empregador").alias("id_cnae"),
        col("cnae_empregador").alias("codigo"),
        col("cnae_empregador_descricao").alias("descricao")
    )
    
    return save_dimension(
        df_cnae,
        "cnae",
        "id_cnae",
        ["codigo", "descricao"],
        conn_params
    )


def create_dim_cid10(df_silver):
    df_cid = df_silver.select(
        col("cid_10").alias("id_cid10"),
        col("cid_10").alias("codigo"),
        col("cid_10").alias("descricao")
    )
    
    return save_dimension(
        df_cid,
        "cid10",
        "id_cid10",
        ["codigo", "descricao"],
        conn_params
    )

### Dimensão Município

A dimensão município é utilizada para:
- local do acidente
- local do empregador

In [10]:
def create_dim_municipio(df_silver):
    df_mun_acidente = df_silver.select(
        col("uf_municipio_acidente").alias("codigo_ibge"),
        col("uf_municipio_acidente").alias("nome"),
        col("uf_municipio_acidente").alias("uf")
    )
    
    df_mun_empregador = df_silver.select(
        col("municipio_empregador").alias("codigo_ibge"),
        col("municipio_empregador").alias("nome"),
        col("uf_municipio_empregador").alias("uf")
    )
    
    df_municipio = df_mun_acidente.union(df_mun_empregador) \
        .distinct() \
        .dropna(subset=["codigo_ibge"])
    
    return save_dimension(
        df_municipio,
        "municipio",
        "codigo_ibge",
        ["nome", "uf"],
        conn_params
    )

### Dimensões de Caracterização do Acidente

Estas dimensões descrevem o contexto e as causas do acidente de trabalho.


In [11]:
def create_dim_tipo_acidente(df_silver):    
    df_tipo_acidente = df_silver.select(
        col("tipo_acidente").alias("id_tipo_acidente"),
        col("tipo_acidente").alias("descricao")
    )
    
    return save_dimension(
        df_tipo_acidente,
        "tipo_acidente",
        "id_tipo_acidente",
        ["descricao"],
        conn_params
    )


def create_dim_lesao(df_silver):
    df_lesao = df_silver.select(
        concat_ws("_", 
                  col("natureza_lesao"), 
                  col("parte_corpo_atingida")
        ).alias("id_lesao"),
        col("natureza_lesao").alias("natureza_lesao"),
        col("parte_corpo_atingida").alias("parte_corpo_atingida")
    )
    
    return save_dimension(
        df_lesao,
        "lesao",
        "id_lesao",
        ["natureza_lesao", "parte_corpo_atingida"],
        conn_params
    )


def create_dim_agente_causador(df_silver):
    df_agente = df_silver.select(
        col("agente_causador_acidente").alias("id_agente_causador"),
        col("agente_causador_acidente").alias("descricao")
    )
    
    return save_dimension(
        df_agente,
        "agente_causador",
        "id_agente_causador",
        ["descricao"],
        conn_params
    )

## Execução da Carga das Dimensões

Nesta etapa é executado o processo completo de criação e carga das dimensões,
respeitando a ordem de dependência entre elas.

In [12]:
try:
    dim_tempo = create_dim_tempo(df_silver)          
    dim_cbo = create_dim_cbo(df_silver)            
    dim_municipio = create_dim_municipio(df_silver)      
    dim_cnae = create_dim_cnae(df_silver)           
    dim_tipo_acidente = create_dim_tipo_acidente(df_silver)  
    dim_lesao = create_dim_lesao(df_silver)          
    dim_agente_causador = create_dim_agente_causador(df_silver)
    dim_cid10 = create_dim_cid10(df_silver)          

    dim_trabalhador = create_dim_trabalhador(df_silver)
    dim_empregador = create_dim_empregador(df_silver)
        
    print("Dimensões foram carregadas")
        
except Exception as e:
    print(f"\nERRO NA CARGA DAS DIMENSÕES: {e}")
    raise e


CARGA DAS DIMENSÕES - STAR SCHEMA

📅 Criando Dimensão Tempo

---> Criando e carregando Dimensão: gold.dim_tempo


26/01/17 15:15:06 WARN TaskSetManager: Stage 1 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
26/01/17 15:15:10 WARN TaskSetManager: Stage 2 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
26/01/17 15:15:11 WARN TaskSetManager: Stage 3 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
26/01/17 15:15:15 WARN TaskSetManager: Stage 13 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.


     Registros únicos: 18083


26/01/17 15:15:16 WARN TaskSetManager: Stage 14 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
26/01/17 15:15:17 WARN TaskSetManager: Stage 15 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

Inserção concluída


...

26/01/17 15:16:28 WARN TaskSetManager: Stage 119 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
26/01/17 15:16:30 WARN TaskSetManager: Stage 125 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.


     Registros únicos: 39659


Inserção concluída


/tmp/ipykernel_24707/2498911532.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, conn)


Dimensão carregada. Registros confirmados: 39659

DIMENSÕES FORAM CARREGADAS


### Preparando dados da camada Silver

In [13]:
df_base = df_silver.select(
    concat_ws("_",
        col("data_acidente_referencia"),
        col("sexo"),
        col("cbo_codigo_descricao"),
        col("cnae_empregador"),
        col("municipio_empregador")
    ).alias("id_cat"),
    
    col("data_acidente_referencia").alias("data_acidente"),
    col("data_emissao_cat").alias("data_emissao"),
    col("data_nascimento"),
    
    concat_ws("_", 
        upper(trim(col("sexo"))),
        col("cbo_codigo_descricao"),
        col("data_nascimento")
    ).alias("id_trabalhador"),
    
    col("cbo_codigo_descricao").alias("codigo_cbo"),
    
    concat_ws("_", 
        col("cnae_empregador"),
        col("municipio_empregador")
    ).alias("id_empregador"),
    
    col("cnae_empregador").alias("codigo_cnae"),
    
    col("uf_municipio_acidente").alias("codigo_municipio_acidente"),
    col("municipio_empregador").alias("codigo_municipio_empregador"),
    
    col("tipo_acidente").alias("codigo_tipo_acidente"),
    col("natureza_lesao").alias("codigo_natureza_lesao"),
    col("parte_corpo_atingida").alias("codigo_parte_corpo"),
    col("agente_causador_acidente").alias("codigo_agente_causador"),
    col("cid_10").alias("codigo_cid10"),
    
    (year(col("data_acidente_referencia")) - year(col("data_nascimento"))).cast(IntegerType()).alias("idade_trabalhador"),
    
    when(col("tipo_acidente").like("%trajeto%"), 1).otherwise(0).alias("flag_trajeto")
)

print(f"Registros preparados: {df_base.count()}")


26/01/17 15:16:37 WARN TaskSetManager: Stage 131 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
[Stage 131:==========================================>              (6 + 2) / 8]

Registros preparados: 291776


## Lookup de Surrogate Keys

Realiza joins (lookups) entre df_base e todas as dimensões
para substituir business keys por surrogate keys (FKs).

- Trocar valores de negócio por IDs técnicos (surrogate keys)
- Criar relacionamentos entre fato e dimensões
- Preparar estrutura final da tabela fato

In [15]:
df_fact = df_base.join(
    dim_tempo.select(
        col("id_tempo").alias("fk_tempo_acidente"),
        col("chv_tempo_org").alias("data_join_acidente")
    ),
    df_base.data_acidente == col("data_join_acidente"),
    "left"
).drop("data_join_acidente")

df_fact = df_fact.join(
    dim_tempo.select(
        col("id_tempo").alias("fk_tempo_emissao"),
        col("chv_tempo_org").alias("data_join_emissao")
    ),
    df_fact.data_emissao == col("data_join_emissao"),
    "left"
).drop("data_join_emissao")

df_fact = df_fact.join(
    dim_tempo.select(
        col("id_tempo").alias("fk_tempo_nascimento"),
        col("chv_tempo_org").alias("data_join_nascimento")
    ),
    df_fact.data_nascimento == col("data_join_nascimento"),
    "left"
).drop("data_join_nascimento")

df_fact = df_fact.join(
    dim_trabalhador.select(
        col("id_trabalhador").alias("fk_trabalhador"),
        col("chv_trabalhador_org").alias("id_trab_join")
    ),
    df_fact.id_trabalhador == col("id_trab_join"),
    "left"
).drop("id_trab_join")

df_fact = df_fact.join(
    dim_cbo.select(
        col("id_cbo").alias("fk_cbo"),
        col("chv_cbo_org").alias("codigo_cbo_join")
    ),
    df_fact.codigo_cbo == col("codigo_cbo_join"),
    "left"
).drop("codigo_cbo_join")

df_fact = df_fact.join(
    dim_empregador.select(
        col("id_empregador").alias("fk_empregador"),
        col("chv_empregador_org").alias("id_emp_join")
    ),
    df_fact.id_empregador == col("id_emp_join"),
    "left"
).drop("id_emp_join")

df_fact = df_fact.join(
    dim_cnae.select(
        col("id_cnae").alias("fk_cnae"),
        col("chv_cnae_org").alias("codigo_cnae_join")
    ),
    df_fact.codigo_cnae == col("codigo_cnae_join"),
    "left"
).drop("codigo_cnae_join")

df_fact = df_fact.join(
    dim_municipio.select(
        col("id_municipio").alias("fk_municipio_acidente"),
        col("chv_municipio_org").alias("mun_acidente_join")
    ),
    df_fact.codigo_municipio_acidente == col("mun_acidente_join"),
    "left"
).drop("mun_acidente_join")

df_fact = df_fact.join(
    dim_municipio.select(
        col("id_municipio").alias("fk_municipio_empregador"),
        col("chv_municipio_org").alias("mun_empregador_join")
    ),
    df_fact.codigo_municipio_empregador == col("mun_empregador_join"),
    "left"
).drop("mun_empregador_join")

df_fact = df_fact.join(
    dim_tipo_acidente.select(
        col("id_tipo_acidente").alias("fk_tipo_acidente"),
        col("chv_tipo_acidente_org").alias("tipo_acidente_join")
    ),
    df_fact.codigo_tipo_acidente == col("tipo_acidente_join"),
    "left"
).drop("tipo_acidente_join")

df_fact = df_fact.withColumn(
    "lesao_join_key",
    concat_ws("_", col("codigo_natureza_lesao"), col("codigo_parte_corpo"))
)

df_fact = df_fact.join(
    dim_lesao.select(
        col("id_lesao").alias("fk_lesao"),
        col("chv_lesao_org").alias("lesao_join")
    ),
    df_fact.lesao_join_key == col("lesao_join"),
    "left"
).drop("lesao_join", "lesao_join_key")

df_fact = df_fact.join(
    dim_agente_causador.select(
        col("id_agente_causador").alias("fk_agente_causador"),
        col("chv_agente_causador_org").alias("agente_join")
    ),
    df_fact.codigo_agente_causador == col("agente_join"),
    "left"
).drop("agente_join")

df_fact = df_fact.join(
    dim_cid10.select(
        col("id_cid10").alias("fk_cid10"),
        col("chv_cid10_org").alias("cid_join")
    ),
    df_fact.codigo_cid10 == col("cid_join"),
    "left"
).drop("cid_join")

print("Lookups concluídos!")


     [1/14] dim_tempo -> data_acidente
     [2/14] dim_tempo -> data_emissao
     [3/14] dim_tempo -> data_nascimento
     [4/14] dim_trabalhador
     [5/14] dim_cbo
     [6/14] dim_empregador
     [7/14] dim_cnae
     [8/14] dim_municipio -> acidente
     [9/14] dim_municipio -> empregador
     [10/14] dim_tipo_acidente
     [11/14] dim_lesao
     [12/14] dim_agente_causador
     [13/14] dim_cid10
     [14/14] Lookups concluídos!


## Montagem da Tabela Fato

In [16]:
df_fact_final = df_fact.select(
    col("id_cat").alias("chv_cat_org"),
    
    "fk_tempo_acidente",
    "fk_tempo_emissao",
    "fk_tempo_nascimento",
    
    "fk_trabalhador",
    "fk_cbo",
    "fk_empregador",
    "fk_cnae",
    
    "fk_municipio_acidente",
    "fk_municipio_empregador",

    "fk_tipo_acidente",
    "fk_lesao",
    "fk_agente_causador",
    "fk_cid10",

    "idade_trabalhador",

    "flag_trajeto"
    
).distinct()

count_fact = df_fact_final.count()
print(f"Registros na tabela fato: {count_fact}")


26/01/17 15:29:24 WARN TaskSetManager: Stage 136 contains a task of very large size (1526 KiB). The maximum recommended task size is 1000 KiB.
26/01/17 15:29:26 WARN TaskSetManager: Stage 144 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
[Stage 169:>                                                        (0 + 8) / 8]

Registros na tabela fato: 145870


### Carga da Tabela Fato

In [17]:
colunas = df_fact_final.columns
fact_table_name = f"{GOLD_SCHEMA}.fato_acidente_trabalho"

print(f"Colunas: {len(colunas)}")
print(f"Colunas a inserir: {colunas}")
print(f"\nPreparando dados para inserção...")

dados_para_inserir = [tuple(row) for row in df_fact_final.collect()]

conn = None
cursor = None

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    query = f"INSERT INTO {fact_table_name} ({', '.join(colunas)}) VALUES %s"
    
    extras.execute_values(
        cursor,
        query,
        dados_para_inserir,
        template=None,
        page_size=1000
    )
    
    conn.commit()
    print("Registros concluída com sucesso!")
except Exception as e:
    print(f"\n❌ Erro ao inserir dados: {e}")
    if conn:
        conn.rollback()
    raise e
    
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    spark.stop()

Colunas: 16
Colunas a inserir: ['chv_cat_org', 'fk_tempo_acidente', 'fk_tempo_emissao', 'fk_tempo_nascimento', 'fk_trabalhador', 'fk_cbo', 'fk_empregador', 'fk_cnae', 'fk_municipio_acidente', 'fk_municipio_empregador', 'fk_tipo_acidente', 'fk_lesao', 'fk_agente_causador', 'fk_cid10', 'idade_trabalhador', 'flag_trajeto']

Preparando dados para inserção...


26/01/17 15:30:09 WARN TaskSetManager: Stage 176 contains a task of very large size (1526 KiB). The maximum recommended task size is 1000 KiB.
26/01/17 15:30:11 WARN TaskSetManager: Stage 181 contains a task of very large size (9331 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

Total de registros a inserir: 145870

🔌 Estabelecendo conexão com PostgreSQL...
💾 Iniciando inserção em lote usando execute_values...
✅ Carga de 145870 registros concluída com sucesso!

🔍 Validando carga...
Registros confirmados no banco: 145870
✅ Validação concluída - carga bem-sucedida!

📈 Estatísticas da tabela fato:
Total de Registros: 145870
Total de Trabalhadores Distintos: 132473
Total de Empregadores Distintos: 39659
Média Idade Trabalhador: 36.58
Total Acidentes de Trajeto: 0

🔌 Conexão com PostgreSQL encerrada.
